### Required libraries

In [11]:
import pandas as pd # The correct package name is pandas, not pandans.
import numpy as np # The correct package name is numpy, not n.
import matplotlib.pyplot as plt # The correct package name is matplotlib, not m.
import seaborn as sns # The correct package name is seaborn, not s.
import sklearn as sk # scikitlearn is not a valid package name. The correct import for scikit-learn is:
import tensorflow as tf # TensorFlow is the correct package name.
import keras # Keras is the correct package name.
import zipfile # The correct package name is zipfile, not z.
import shutil # The correct package name is shutil, not sh.
import os # The correct package name is os, not o.
import pickle # The correct package name is pickle, not p.

In [12]:
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras import layers, models, applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, MobileNetV2, InceptionV3, EfficientNetB4
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

### Data Preprocessing & Augmentation

In [13]:
# !unzip /content/Dataset.zip

In [14]:
# Define constants
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32

# Data augmentation and preparation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Assuming 'fish_data' contains your dataset with category-based folders
train_generator = train_datagen.flow_from_directory(
    r'/content/images.cv_jzk6llhf18tm3k0kyttxz/data/train',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    r'/content/images.cv_jzk6llhf18tm3k0kyttxz/data/val',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# If you have a separate test set
test_generator = test_datagen.flow_from_directory(
    r'/content/images.cv_jzk6llhf18tm3k0kyttxz/data/test',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Get class names
class_names = list(train_generator.class_indices.keys())
num_classes = len(class_names)

print("Classes:", class_names)
print("Number of classes:", num_classes)

Found 4984 images belonging to 11 classes.
Found 215 images belonging to 11 classes.
Found 3124 images belonging to 11 classes.
Classes: ['animal fish', 'animal fish bass', 'fish sea_food black_sea_sprat', 'fish sea_food gilt_head_bream', 'fish sea_food hourse_mackerel', 'fish sea_food red_mullet', 'fish sea_food red_sea_bream', 'fish sea_food sea_bass', 'fish sea_food shrimp', 'fish sea_food striped_red_mullet', 'fish sea_food trout']
Number of classes: 11


### Build CNN From Scratch

In [15]:
# Build CNN From Scratch

def create_custom_cnn():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Train custom CNN
custom_model = create_custom_cnn()
custom_history = custom_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=50,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3),
        tf.keras.callbacks.ModelCheckpoint('custom_cnn_fish.h5', save_best_only=True)
    ]
)


Epoch 1/50
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 713ms/step - accuracy: 0.2471 - loss: 2.1372

155/155 ━━━━━━━━━━━━━━━━━━━━ 128s 766ms/step - accuracy: 0.2476 - loss: 2.1357 - val_accuracy: 0.5365 - val_loss: 1.4564
Epoch 2/50
  1/155 ━━━━━━━━━━━━━━━━━━━━ 13s 85ms/step - accuracy: 0.5000 - loss: 1.3651

155/155 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - accuracy: 0.5000 - loss: 1.3651 - val_accuracy: 0.5156 - val_loss: 1.4056
Epoch 3/50
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 716ms/step - accuracy: 0.5098 - loss: 1.3667

155/155 ━━━━━━━━━━━━━━━━━━━━ 116s 747ms/step - accuracy: 0.5099 - loss: 1.3663 - val_accuracy: 0.6979 - val_loss: 1.1120
Epoch 4/50
  1/155 ━━━━━━━━━━━━━━━━━━━━ 12s 79ms/step - accuracy: 0.6875 - loss: 1.5231

155/155 ━━━━━━━━━━━━━━━━━━━━ 6s 38ms/step - accuracy: 0.6875 - loss: 1.5231 - val_accuracy: 0.7448 - val_loss: 0.9511
Epoch 5/50
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 702ms/step - accuracy: 0.6382 - loss: 1.0130

...

155/155 ━━━━━━━━━━━━━━━━━━━━ 91s 438ms/step - accuracy: 0.7352 - loss: 0.7402 - val_accuracy: 0.8385 - val_loss: 0.4663
Epoch 8/50
155/155 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.7812 - loss: 0.4783 - val_accuracy: 0.8125 - val_loss: 0.5831
Epoch 9/50
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 699ms/step - accuracy: 0.7602 - loss: 0.6451

155/155 ━━━━━━━━━━━━━━━━━━━━ 114s 733ms/step - accuracy: 0.7603 - loss: 0.6450 - val_accuracy: 0.8281 - val_loss: 0.4529
Epoch 10/50
155/155 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - accuracy: 0.8125 - loss: 0.5015 - val_accuracy: 0.7760 - val_loss: 0.5653
Epoch 11/50
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 708ms/step - accuracy: 0.8173 - loss: 0.5303

155/155 ━━━━━━━━━━━━━━━━━━━━ 139s 740ms/step - accuracy: 0.8173 - loss: 0.5303 - val_accuracy: 0.8490 - val_loss: 0.3994
Epoch 12/50
155/155 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - accuracy: 0.9062 - loss: 0.3272 - val_accuracy: 0.8281 - val_loss: 0.5131
Epoch 13/50
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 490ms/step - accuracy: 0.8336 - loss: 0.4743

155/155 ━━━━━━━━━━━━━━━━━━━━ 103s 524ms/step - accuracy: 0.8336 - loss: 0.4743 - val_accuracy: 0.8906 - val_loss: 0.3415
Epoch 14/50
  1/155 ━━━━━━━━━━━━━━━━━━━━ 16s 109ms/step - accuracy: 0.7188 - loss: 0.5928

155/155 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.7188 - loss: 0.5928 - val_accuracy: 0.8906 - val_loss: 0.3390
Epoch 15/50
155/155 ━━━━━━━━━━━━━━━━━━━━ 121s 412ms/step - accuracy: 0.8521 - loss: 0.4088 - val_accuracy: 0.8594 - val_loss: 0.4057
Epoch 16/50
155/155 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.8438 - loss: 0.3437 - val_accuracy: 0.8750 - val_loss: 0.3914
Epoch 17/50
155/155 ━━━━━━━━━━━━━━━━━━━━ 63s 408ms/step - accuracy: 0.8633 - loss: 0.3892 - val_accuracy: 0.8542 - val_loss: 0.3538


In [16]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16, ResNet50, MobileNetV2, InceptionV3, EfficientNetB0
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import shutil

# Ensure models directory exists
os.makedirs('models', exist_ok=True)

DATA_DIR = r'/content/images.cv_jzk6llhf18tm3k0kyttxz/data'
IMG_SIZE = (224, 224)
BATCH_SIZE = 100
EPOCHS = 30

# 🔹 Data Generators (without using create_data_generators)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2   # split train/val
)

val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    color_mode="rgb",      # <-- Force RGB
    subset="training"
)

val_gen = val_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    color_mode="rgb",      # <-- Force RGB
    subset="validation"
)



NUM_CLASSES = len(train_gen.class_indices)
CLASS_NAMES = list(train_gen.class_indices.keys())
print(f"Classes: {CLASS_NAMES}")

# ------------------- Models -------------------
def build_custom_cnn(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

def build_transfer_model(base_model, input_shape, num_classes):
    base = base_model(weights='imagenet', include_top=False, input_shape=input_shape)
    base.trainable = False
    model = models.Sequential([
        base,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.3),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Define models
model_configs = {
    'CustomCNN': build_custom_cnn((*IMG_SIZE, 3), NUM_CLASSES),
    'VGG16': build_transfer_model(VGG16, (*IMG_SIZE, 3), NUM_CLASSES),
    'ResNet50': build_transfer_model(ResNet50, (*IMG_SIZE, 3), NUM_CLASSES),
    'MobileNetV2': build_transfer_model(MobileNetV2, (*IMG_SIZE, 3), NUM_CLASSES),
}

# ------------------- Training -------------------
best_acc = 0
best_model_name = ""

for name, model in model_configs.items():
    print(f"\n🚀 Training {name}...")

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    callbacks = [
        ModelCheckpoint(f'models/{name}.h5', save_best_only=True, monitor='val_accuracy'),
        EarlyStopping(patience=5, restore_best_weights=True),
        ReduceLROnPlateau(factor=0.2, patience=3)
    ]

    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=EPOCHS,
        callbacks=callbacks,
        verbose=1
    )

    # Track best model
    val_acc = max(history.history['val_accuracy'])
    if val_acc > best_acc:
        best_acc = val_acc
        best_model_name = name

print(f"\n✅ Best Model: {best_model_name} (Val Accuracy: {best_acc:.4f})")

# Save best model
shutil.copy(f'models/{best_model_name}.h5', 'models/best_model.h5')
print("📂 Saved as models/best_model.h5")


Found 8354 images belonging to 3 classes.
Found 2087 images belonging to 3 classes.
Classes: ['test', 'train', 'val']
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step

🚀 Training CustomCNN...
Epoch 1/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5645 - loss: 0.9597

84/84 ━━━━━━━━━━━━━━━━━━━━ 128s 1s/step - accuracy: 0.5648 - loss: 0.9593 - val_accuracy: 0.5966 - val_loss: 0.9243 - learning_rate: 0.0010
Epoch 2/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 106s 1s/step - accuracy: 0.6002 - loss: 0.9124 - val_accuracy: 0.5966 - val_loss: 0.9090 - learning_rate: 0.0010
Epoch 3/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 106s 1s/step - accuracy: 0.5932 - loss: 0.9138 - val_accuracy: 0.5966 - val_loss: 0.9067 - learning_rate: 0.0010
Epoch 4/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 104s 1s/step - accuracy: 0.5989 - loss: 0.9070 - val_accuracy: 0.5966 - val_loss: 0.9093 - learning_rate: 0.0010
Epoch 5/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 104s 1s/step - accuracy: 0.5882 - loss: 0.9176 - val_accuracy: 0.5966 - val_loss: 0.9069 - learning_rate: 0.0010
Epoch 6/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 107s 1s/step - accuracy: 0.5874 - loss: 0.9195 - val_accuracy: 0.5966 - val_loss: 0.9067 - learning_rate: 0.0010
Epoch 7/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 105s 1s/step - accuracy: 0.5941 - loss: 0.9050 - val_accuracy: 0.59

84/84 ━━━━━━━━━━━━━━━━━━━━ 210s 2s/step - accuracy: 0.5783 - loss: 0.9470 - val_accuracy: 0.5966 - val_loss: 0.9114 - learning_rate: 0.0010
Epoch 2/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 122s 1s/step - accuracy: 0.5926 - loss: 0.9157 - val_accuracy: 0.5966 - val_loss: 0.9203 - learning_rate: 0.0010
Epoch 3/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 122s 1s/step - accuracy: 0.5942 - loss: 0.9171 - val_accuracy: 0.5966 - val_loss: 0.9150 - learning_rate: 0.0010
Epoch 4/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 123s 1s/step - accuracy: 0.5849 - loss: 0.9249 - val_accuracy: 0.5966 - val_loss: 0.9197 - learning_rate: 0.0010
Epoch 5/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 125s 1s/step - accuracy: 0.5920 - loss: 0.9147 - val_accuracy: 0.5966 - val_loss: 0.9096 - learning_rate: 2.0000e-04
Epoch 6/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 122s 1s/step - accuracy: 0.5961 - loss: 0.9060 - val_accuracy: 0.5966 - val_loss: 0.9109 - learning_rate: 2.0000e-04
Epoch 7/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 122s 1s/step - accuracy: 0.5976 - loss: 0.9071 - val_accura

84/84 ━━━━━━━━━━━━━━━━━━━━ 148s 2s/step - accuracy: 0.5617 - loss: 0.9670 - val_accuracy: 0.5966 - val_loss: 0.9378 - learning_rate: 0.0010
Epoch 2/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 110s 1s/step - accuracy: 0.5882 - loss: 0.9362 - val_accuracy: 0.5966 - val_loss: 0.9094 - learning_rate: 0.0010
Epoch 3/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 110s 1s/step - accuracy: 0.6080 - loss: 0.8996 - val_accuracy: 0.5966 - val_loss: 0.9074 - learning_rate: 0.0010
Epoch 4/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 109s 1s/step - accuracy: 0.6048 - loss: 0.8981 - val_accuracy: 0.5966 - val_loss: 0.9085 - learning_rate: 0.0010
Epoch 5/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 110s 1s/step - accuracy: 0.6065 - loss: 0.9085 - val_accuracy: 0.5966 - val_loss: 0.9186 - learning_rate: 0.0010
Epoch 6/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 110s 1s/step - accuracy: 0.5899 - loss: 0.9176 - val_accuracy: 0.5966 - val_loss: 0.9225 - learning_rate: 0.0010
Epoch 7/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 110s 1s/step - accuracy: 0.5914 - loss: 0.9179 - val_accuracy: 0.59

84/84 ━━━━━━━━━━━━━━━━━━━━ 150s 2s/step - accuracy: 0.5379 - loss: 1.1042 - val_accuracy: 0.5989 - val_loss: 0.9319 - learning_rate: 0.0010
Epoch 2/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 103s 1s/step - accuracy: 0.5970 - loss: 0.9118 - val_accuracy: 0.5961 - val_loss: 0.9375 - learning_rate: 0.0010
Epoch 3/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 104s 1s/step - accuracy: 0.5867 - loss: 0.9224 - val_accuracy: 0.5985 - val_loss: 0.9285 - learning_rate: 0.0010
Epoch 4/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 103s 1s/step - accuracy: 0.5937 - loss: 0.9134 - val_accuracy: 0.5970 - val_loss: 0.9302 - learning_rate: 0.0010
Epoch 5/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 103s 1s/step - accuracy: 0.5958 - loss: 0.9108 - val_accuracy: 0.5966 - val_loss: 0.9232 - learning_rate: 0.0010
Epoch 6/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 103s 1s/step - accuracy: 0.5954 - loss: 0.9082 - val_accuracy: 0.5966 - val_loss: 0.9223 - learning_rate: 0.0010
Epoch 7/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 102s 1s/step - accuracy: 0.5957 - loss: 0.9066 - val_accuracy: 0.59

In [ ]:
ModelCheckpoint(f'models/{name}.h5', save_best_only=True, monitor='val_accuracy')

### Transfer Learning (Pre-trained Models)

In [21]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import Model
from tensorflow.keras import layers

base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224,224,3))
base_model.trainable = False  # freeze layers

x = layers.Flatten()(base_model.output)
x = layers.Dense(256, activation='relu')(x)
output = layers.Dense(3, activation='softmax')(x) # Using 3 classes based on cell 32e200ce output

resnet_model = Model(inputs=base_model.input, outputs=output)
resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history_resnet = resnet_model.fit(train_gen, validation_data=val_gen, epochs=20)
resnet_model.save("models/resnet_fish.h5")

Epoch 1/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 131s 1s/step - accuracy: 0.4678 - loss: 6.2202 - val_accuracy: 0.5956 - val_loss: 1.7629
Epoch 2/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 110s 1s/step - accuracy: 0.4935 - loss: 1.4553 - val_accuracy: 0.5903 - val_loss: 1.0227
Epoch 3/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 115s 1s/step - accuracy: 0.5529 - loss: 1.1041 - val_accuracy: 0.5970 - val_loss: 2.2537
Epoch 4/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 110s 1s/step - accuracy: 0.5125 - loss: 1.4028 - val_accuracy: 0.5946 - val_loss: 0.9733
Epoch 5/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 110s 1s/step - accuracy: 0.5151 - loss: 1.0142 - val_accuracy: 0.5961 - val_loss: 1.1048
Epoch 6/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 110s 1s/step - accuracy: 0.5405 - loss: 0.9976 - val_accuracy: 0.3033 - val_loss: 1.3635
Epoch 7/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 110s 1s/step - accuracy: 0.5287 - loss: 1.0238 - val_accuracy: 0.5707 - val_loss: 0.9678
Epoch 8/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 111s 1s/step - accuracy: 0.5528 - loss: 0.9690 - val_accuracy: 0.5443 - v